選手のレース場、着順、日付をひたすらまとめていく

In [4]:
#!pip install tqdm

In [28]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm_notebook as tqdm

#格納したいカラムのデータフレームを作成する
Player_Data = ["PlayerID", "PlayerName", "Rank", "Day", "Race_No", "Place"]
Player_Data_Ghost = pd.DataFrame(index=[], columns=Player_Data)
df_player_data = pd.DataFrame(index=[], columns=Player_Data)

In [5]:
result_file_list = glob("test/*/result_html/*")
result_file = [x.replace("\\", "/") for x in result_file_list]
#print(result_file[1])

#途中から始める処理を書く
is_file = os.path.isfile("match_up.txt")

target_txts_list = [] #配列を空にする
match_up_list = [] #配列を空にする

if is_file:
    with open("match_up.txt", "r") as tf:
        match_up_list = tf.read().split(',')[:-1]
    #途中から始めるためのコード書く
    target_txts_list = set(result_file) - set(match_up_list)
else:
    target_txts_list = result_file

print(len(result_file))
print(len(target_txts_list))

#ディレクトリ探す
#jou_number = os.listdir(f"test/")


37474
37474


In [6]:
#result_htmlにアクセスして回す

for target_txt in tqdm(target_txts_list):
    #レース結果を保存したhtmlファイルとそのDF
    dfs = pd.read_html(target_txt, encoding="utf-8")

    tyakujun = dfs[2]
    Player_Data_Ghost["PlayerID"] = tyakujun["ボートレーサー"].str.extract(r"(\d+)").astype(int)
    Player_Data_Ghost["PlayerName"] = tyakujun["ボートレーサー"].str.extract(r"(\D+)")
    Player_Data_Ghost["Place"] = target_txt[-19:-17]
    Player_Data_Ghost["Day"] = target_txt[-16:-8]#Data型にしたい
    Player_Data_Ghost["Day"] = pd.to_datetime(Player_Data_Ghost["Day"])
    Player_Data_Ghost["Race_No"] = target_txt[-7:-5]
    Player_Data_Ghost["Rank"] = tyakujun["着"]

    #DFに情報を追加する
    df_player_data = df_player_data.append(Player_Data_Ghost)

    #取得済みリストを作る
    with open("match_up.txt", "a", encoding="utf-8") as file:
            file.write(target_txt + ",")

df_player_data.drop_duplicates(keep="last", inplace=True)


C:\Users\rstta\AppData\Local\Temp/ipykernel_9520/3149970973.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_txt in tqdm(target_txts_list):


  0%|          | 0/37474 [00:00<?, ?it/s]

In [7]:
display(df_player_data)
df_player_data.describe()

,PlayerID,PlayerName,Rank,Day,Race_No,Place
0,4645,上村 純一,１,2014-01-02,01,01
1,3532,柴田 光,２,2014-01-02,01,01
2,3115,鈴木 正人,３,2014-01-02,01,01
3,3560,太田 克哉,４,2014-01-02,01,01
4,3376,萩原 善行,５,2014-01-02,01,01
...,...,...,...,...,...,...
1,4954,内山 峻輔,２,2022-03-18,12,02
2,4360,庄司 孝輔,３,2022-03-18,12,02
3,3584,佐々木 英樹,４,2022-03-18,12,02
4,4532,秋元 哲,５,2022-03-18,12,02


C:\Users\rstta\AppData\Local\Temp/ipykernel_9520/196182690.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_player_data.describe()


,PlayerID,PlayerName,Rank,Day,Race_No,Place
count,224844.0,224844,224843,224844,224844,224844
unique,1879.0,1878,17,2285,12,2
top,3251.0,平石 和男,２,2019-12-03 00:00:00,01,01
freq,617.0,617,37472,144,18816,113154
first,NaN,NaN,NaN,2014-01-02 00:00:00,NaN,NaN
last,NaN,NaN,NaN,2022-03-18 00:00:00,NaN,NaN


In [11]:
df_player_data.isna().sum()

PlayerID      0
PlayerName    0
Rank          1
Day           0
Race_No       0
Place         0
dtype: int64

In [8]:
output_file = "PlayerRankData.csv"
df_player_data.to_csv(os.path.join("data/", output_file), mode="a", index=False)

# 目的は？四半期、半期毎のトータルの複勝率、三連率、開催地の複勝率、三連率をリターンする関数

In [6]:
PlayerRankData = pd.read_csv("data/PlayerRankData.csv", encoding="utf-8")

In [20]:
import pandas as pd
import datetime

PlayerRankData = pd.read_csv("data/PlayerRankData.csv", encoding="utf-8")

def player_win_rate():
#NoAは出場数、_2は二連率、_3は3連率、Venueは開催地、qは3ヶ月、hは6ヶ月
Player_Win = [ "RaceID", "PlayerID",
                "q_Total_2", "q_Total_3", "h_Total_2", "h_Total_3",
                "q_Venue_2", "q_Venue_3", "h_Venue_2", "h_Venue_3"]

Player_Win_Rate = pd.DataFrame(index=[], columns=Player_Win)

## 変数
#レースID
race_id = "012015060101"

#開催日、半期、四半期
open_day = datetime.datetime.strptime("2015-06-01", '%Y-%m-%d') #Dayの値を持ってきたい
bf_h_day = open_day - datetime.timedelta(days=180)
bf_q_day = open_day - datetime.timedelta(days=90)

open_day = str(open_day.date())
bf_h_day = str(bf_h_day.date())
bf_q_day = str(bf_q_day.date())

#開催地
open_place = "1"
#プレイヤー
player_id = 4045

## 期間と開催地を絞る

PRD_H_Period = PlayerRankData.query(f"'{bf_h_day}' <= Day <= '{open_day}'")#開催日から半期さかのぼる処理を書きたい
PRD_Q_Period = PlayerRankData.query(f'"{bf_q_day}" <= Day <= "{open_day}"')#開催日から4半期さかのぼる処理を書きたい

#開催地のデータ―
#Placeを変数にする
PRD_Loc_H_Period = PRD_H_Period.query(f"{open_place} == Place") #半期の開催地のデータ
PRD_Loc_Q_Period = PRD_Q_Period.query(f"{open_place} == Place") #四半期の開催地のデータ

#PlayerIDの選手の全出場数

Player_Turn_Sum = PRD_H_Period[PRD_H_Period["PlayerID"] == player_id]
Player_Loc_Turn_Sum = PRD_Loc_H_Period[PRD_Loc_H_Period["PlayerID"] == player_id]
#PlayerIDの選手の開催地の出場数
Player_Q_Turn_Sum = PRD_Q_Period[PRD_Q_Period["PlayerID"] == player_id]
Player_Q_Loc_Turn_Sum = PRD_Loc_Q_Period[PRD_Loc_Q_Period["PlayerID"] == player_id]

#出場回数によって決められた値を与える
#半期の全出場回数が200未満はnanをとりあえず入れる。
#後で平均値よりしたの値を代入

if len(Player_Turn_Sum) >= 0:
    # 半期毎
    ## 全開催地の勝率を計算
    PRD_H_1st = PRD_H_Period['PlayerID'][PRD_H_Period['Rank']=="１"].value_counts() / PRD_H_Period['PlayerID'].value_counts()
    PRD_H_2nd = PRD_H_Period['PlayerID'][PRD_H_Period['Rank']=="２"].value_counts() / PRD_H_Period['PlayerID'].value_counts()
    PRD_H_3rd = PRD_H_Period['PlayerID'][PRD_H_Period['Rank']=="３"].value_counts() / PRD_H_Period['PlayerID'].value_counts()

    #DW = 複勝率, TW = 三連率
    DW_H_Rate = PRD_H_1st + PRD_H_2nd
    DW_H_Rate = DW_H_Rate.fillna(0)
    TW_H_Rate = DW_H_Rate + PRD_H_3rd
    TW_H_Rate = TW_H_Rate.fillna(0)

    Player_Win_Rate.loc[1,"h_Total_2"] = DW_H_Rate[player_id]
    Player_Win_Rate.loc[1,"h_Total_3"] = TW_H_Rate[player_id]

else:
    Player_Win_Rate.loc[1,"h_Total_2"] = None
    Player_Win_Rate.loc[1,"h_Total_3"] = None



if len(Player_Loc_Turn_Sum) > 0:
    ##開催地の勝率を計算
    PRD_Loc_H_1st = PRD_Loc_H_Period['PlayerID'][PRD_Loc_H_Period['Rank']=="１"].value_counts() / PRD_Loc_H_Period['PlayerID'].value_counts()
    PRD_Loc_H_2nd = PRD_Loc_H_Period['PlayerID'][PRD_Loc_H_Period['Rank']=="２"].value_counts() / PRD_Loc_H_Period['PlayerID'].value_counts()
    PRD_Loc_H_3rd = PRD_Loc_H_Period['PlayerID'][PRD_Loc_H_Period['Rank']=="３"].value_counts() / PRD_Loc_H_Period['PlayerID'].value_counts()

    DW_Loc_H_Rate = PRD_Loc_H_1st + PRD_Loc_H_2nd
    DW_Loc_H_Rate = DW_Loc_H_Rate.fillna(0)
    TW_Loc_H_Rate = DW_Loc_H_Rate + PRD_Loc_H_3rd
    TW_Loc_H_Rate = TW_Loc_H_Rate.fillna(0)

    Player_Win_Rate.loc[1,"q_Venue_2"] = DW_Loc_H_Rate[player_id]
    Player_Win_Rate.loc[1,"q_Venue_2"] = TW_Loc_H_Rate[player_id]


else:
    Player_Win_Rate.loc[1,"q_Venue_2"] = None
    Player_Win_Rate.loc[1,"q_Venue_2"] = None


#開催地毎の勝率
if len(Player_Q_Turn_Sum) >= 0:
    #四半期毎
    ##全開催地の勝率を計算
    PRD_Q_1st = PRD_Q_Period['PlayerID'][PRD_Q_Period['Rank']=="１"].value_counts() / PRD_Q_Period['PlayerID'].value_counts()
    PRD_Q_2nd = PRD_Q_Period['PlayerID'][PRD_Q_Period['Rank']=="２"].value_counts() / PRD_Q_Period['PlayerID'].value_counts()
    PRD_Q_3rd = PRD_Q_Period['PlayerID'][PRD_Q_Period['Rank']=="３"].value_counts() / PRD_Q_Period['PlayerID'].value_counts()

    DW_Q_Rate = PRD_Q_1st + PRD_Q_2nd
    DW_Q_Rate = DW_Q_Rate.fillna(0)
    TW_Q_Rate = DW_Q_Rate + PRD_Q_3rd
    TW_Q_Rate = TW_Q_Rate.fillna(0)

    Player_Win_Rate.loc[1,"q_Total_2"] = DW_Q_Rate[player_id]
    Player_Win_Rate.loc[1,"q_Total_3"] = TW_Q_Rate[player_id]

else:
    Player_Win_Rate.loc[1,"q_Total_2"] = None
    Player_Win_Rate.loc[1,"q_Total_3"] = None


if len(Player_Q_Loc_Turn_Sum) > 0:
    ##開催地の勝率を計算
    PRD_Loc_Q_1st = PRD_Loc_Q_Period['PlayerID'][PRD_Loc_Q_Period['Rank']=="１"].value_counts() / PRD_Loc_Q_Period['PlayerID'].value_counts()
    PRD_Loc_Q_2nd = PRD_Loc_Q_Period['PlayerID'][PRD_Loc_Q_Period['Rank']=="２"].value_counts() / PRD_Loc_Q_Period['PlayerID'].value_counts()
    PRD_Loc_Q_3rd = PRD_Loc_Q_Period['PlayerID'][PRD_Loc_Q_Period['Rank']=="３"].value_counts() / PRD_Loc_Q_Period['PlayerID'].value_counts()

    DW_Loc_Q_Rate = PRD_Loc_Q_1st + PRD_Loc_Q_2nd
    DW_Loc_Q_Rate = DW_Loc_Q_Rate.fillna(0)
    TW_Loc_Q_Rate = DW_Loc_Q_Rate + PRD_Loc_Q_3rd
    TW_Loc_Q_Rate = TW_Loc_Q_Rate.fillna(0)

    Player_Win_Rate.loc[1,"q_Venue_2"] = DW_Loc_Q_Rate[player_id]
    Player_Win_Rate.loc[1,"q_Venue_3"] = TW_Loc_Q_Rate[player_id]

else:
    Player_Win_Rate.loc[1,"q_Venue_2"] = None
    Player_Win_Rate.loc[1,"q_Venue_3"] = None


Player_Win_Rate.loc[1,"PlayerID"] = player_id #PlayerIDの変数
Player_Win_Rate.loc[1,"RaceID"] = race_id #PlayerIDの変数



,RaceID,PlayerID,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3
1,012015060101,4045,0.555556,0.0,0.388889,0.444444,None,None,NaN,NaN


In [ ]:
display(PlayerRankData)
PlayerRankData.dtypes

In [ ]:
#player_winrate()
{chr(ord('０')+i):i for i in range(1,7)} #こんな辞書を渡すとrankはintにできそう

In [16]:
import os
import pandas as pd

#NoAは出場数、_2は二連率、_3は3連率、Venueは開催地、qは3ヶ月、hは6ヶ月
Player_Win = [ "RaceID","PlayerID", "NoA",
                "q_Total_2", "q_Total_3", "h_Total_2", "h_Total_3",
                "q_Venue_2", "q_Venue_3", "h_Venue_2", "h_Venue_3"]

url = "test/01/result_html/01_20140102_01.html"
dfs = pd.read_html(url, encoding="utf-8")
dfs[2]

Player_Win_Rate = pd.DataFrame(index=[], columns=Player_Win)
